In [88]:
from sklearn.datasets import fetch_20newsgroups
import nltk.data
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import logging
from gensim.models import Word2Vec,KeyedVectors
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from nltk.stem.porter import PorterStemmer
from stemming.porter2 import stem
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [10]:
#loading the news dataset from sklearn dataset library
twenty_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

# tokenize to sentences based on the notations followed in english literature
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
print("\n".join(twenty_train.data[0].split("\n")[:3]))
print(twenty_train.target_names[twenty_train.target[0]])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
rec.autos


In [11]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(sentence).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ",text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [12]:
# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [15]:
#train_sentences and test_sentences contain the filtered sentences of training and testing data respectively

train_sentences = []  # Initialize an empty list of sentences
test_sentences = []

print "Parsing sentences from training set"
for i in range(len(twenty_train.data)):
    train_sentences += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(twenty_train.data[i], tokenizer)]]
print len(train_sentences)
print train_sentences[0]

print "Parsing sentences from unlabeled set"
for i in range(len(twenty_test.data)):
    test_sentences += [[stem(str(word)) for word in sentence] \
                       for sentence in [text_to_sentences(twenty_test.data[i], tokenizer)]]
print len(test_sentences)
print test_sentences[0]

Parsing sentences from training set
11314
["[u'lerxst', u'wam', u'umd', u'edu', u'thing', u'subject', u'car']", "[u'nntp', u'posting', u'host', u'rac', u'wam', u'umd', u'edu', u'organization', u'university', u'maryland', u'college', u'park', u'lines', u'wondering', u'anyone', u'could', u'enlighten', u'car', u'saw', u'day']", "[u'door', u'sports', u'car', u'looked', u'late', u'early']", "[u'called', u'bricklin']", "[u'doors', u'really', u'small']", "[u'addition', u'front', u'bumper', u'separate', u'rest', u'body']", "[u'know']", "[u'anyone', u'tellme', u'model', u'name', u'engine', u'specs', u'years', u'production', u'car', u'made', u'history', u'whatever', u'info', u'funky', u'looking', u'car', u'please', u'e', u'mail']", "[u'thanks', u'il', u'brought', u'neighborhood', u'lerxst']"]
Parsing sentences from unlabeled set
7532
["[u'v', u'mb', u'k', u'ubvmsd', u'cc', u'buffalo', u'edu', u'neil', u'b', u'gandler', u'subject', u'need', u'info', u'bonneville', u'organization', u'university', 

In [16]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
model = Word2Vec(train_sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)
#model.save_word2vec_format(model_name,binary=False)

2017-03-22 16:53:01,857 : INFO : collecting all words and their counts
2017-03-22 16:53:01,859 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2017-03-22 16:53:02,497 : INFO : PROGRESS: at sentence #10000, processed 166860 words, keeping 130308 word types
2017-03-22 16:53:02,515 : INFO : collected 144031 word types from a corpus of 187884 raw words and 11314 sentences
2017-03-22 16:53:02,516 : INFO : Loading a fresh vocabulary
2017-03-22 16:53:02,616 : INFO : min_count=40 retains 28 unique words (0% of original 144031, drops 144003)
2017-03-22 16:53:02,617 : INFO : min_count=40 leaves 7150 word corpus (3% of original 187884, drops 180734)
2017-03-22 16:53:02,853 : INFO : deleting the raw counts dictionary of 144031 items
2017-03-22 16:53:02,861 : INFO : sample=0.001 downsamples 28 most-common words
2017-03-22 16:53:02,863 : INFO : downsampling leaves estimated 1012 word corpus (14.2% of prior 7150)
2017-03-22 16:53:02,864 : INFO : estimated required memory for 28 words and 300 dimensions: 81200 bytes
2017-03-22 16:53:02,881 : INFO : resetting layer weights
2017-03-22 16:53:02,904 : INFO : training model with 4 workers on 28 v

In [ ]:
#print model.syn0.shape
# model.wv is defined using keyedVectors
print model.wv.most_similar("car")
#print len(model.vocab)

In [103]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float128")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [104]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float128")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [105]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. 

trainDataVecs = getAvgFeatureVecs( train_sentences, model, num_features )
testDataVecs = getAvgFeatureVecs( test_sentences, model, num_features )

Review 0 of 11314
Review 1000 of 11314
Review 2000 of 11314
Review 3000 of 11314


/home/dell/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: invalid value encountered in divide


Review 4000 of 11314
Review 5000 of 11314
Review 6000 of 11314
Review 7000 of 11314
Review 8000 of 11314
Review 9000 of 11314
Review 10000 of 11314
Review 11000 of 11314
Review 0 of 7532
Review 1000 of 7532
Review 2000 of 7532
Review 3000 of 7532
Review 4000 of 7532
Review 5000 of 7532
Review 6000 of 7532
Review 7000 of 7532


In [106]:
#trainDataVecs = np.around(trainDataVecs,decimals=4)
print trainDataVecs[0]

#testDataVecs = np.round(testDataVecs,decimals =4)

# Fit a random forest to the training data, using 100 trees

df_train = pd.DataFrame(trainDataVecs)
df_test = pd.DataFrame(testDataVecs)
train_data = df_train.fillna(lambda x: x.median(),inplace=True)
test_data = df_test.fillna(lambda x: x.median(),inplace=True)
trainDataVecs = np.array(train_data)
testDataVecs = np.array(test_data)
#forest = RandomForestClassifier( n_estimators = 100 )
clf = svm.SVC()
print "Fitting a random forest to labeled training data..."
#forest = forest.fit( trainDataVecs, twenty_train.target )
clf = clf.fit(trainDataVecs,twenty_train.target)
# Test & extract results 
#result = forest.predict( testDataVecs )
result = clf.predict(testDataVecs)
print (np.mean(result == twenty_test.target))


[ 0.0017914181  0.00045829749 -0.0011988715 -0.0014944462  7.9838377e-05
  0.0008076072  0.00055176084 -0.0014577545  0.0020071978  0.0019292455
  0.0012856046 -0.0022874423 -0.00040315054  0.0018680559  0.00099064375
  0.0012036987 -0.0011523856  1.4033991e-05  0.00055946677  0.0011840842
  0.0012771272 -0.00032269457 -0.00051568495  0.0011304863  0.00014196549
 -0.00071088393  0.0010496145 -0.00042162504  0.0013276864 -0.00055119634
 -0.00030553734 -0.0011083321 -0.0020648804  0.0013676631  0.0015148928
 -0.001185488  0.00068430789 -0.00064016914  0.00050516997 -2.1113752e-05
  6.0231068e-05  0.0012498369 -6.6119872e-05 -0.00089587824  0.00055207242
  0.00074021035  0.0018263939  0.0010504036 -0.00034734813  7.5800192e-05
 -0.0012240779  0.0016564614  0.00041699511  0.0017040828  0.0012606456
 -0.00020225953 -0.00022171278 -0.00068254716 -0.002163365  9.2205315e-05
 -0.00016372518  0.00044649004  1.054918e-05  0.0013885001 -0.0018665136
  0.001020532 -0.00037337586 -0.0009401344 -0.0

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').